In [31]:
from datetime import datetime
import logging
import sys
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback

import warnings
warnings.filterwarnings('ignore')

optuna_dir = "..//optuna"

test_df = pd.read_csv(r'..//data//test.csv')
train_df = pd.read_csv(r'..//data//train.csv')
train_extra_df = pd.read_csv(r'..//data//training_extra.csv')

train_df = pd.concat([train_df, train_extra_df], ignore_index=True)

train_df.columns = [
    'id', 'brand', 'material', 'size', 'compartments', 
    'laptop_compartment', 'is_waterproof', 'style', 'color', 
    'weight_capacity', 'price'
]
test_df.columns = [
    'id', 'brand', 'material', 'size', 'compartments', 
    'laptop_compartment', 'is_waterproof', 'style', 'color', 
    'weight_capacity',
]

cat_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

# Convert all to be the same type
train_df[cat_cols] = train_df[cat_cols].astype('category')
test_df[cat_cols] = test_df[cat_cols].astype('category')

target = "price"
X = train_df.drop(columns=[target])
y = train_df[target]

# Only keeping important columns
model_features = [
    'weight_capacity', 'color', 'compartments', 'brand', 'material', 'is_waterproof'
]
X = X[model_features]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        # 'boosting_type': boosting_type,
        'num_leaves': trial.suggest_int('num_leaves', 200, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 5000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 200),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, 1.0),
        'max_bin': trial.suggest_int('max_bin', 50, 1000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.1, 1.0),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    # For the DART boosting type, include dart‑specific parameters.

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=100, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse


optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=100)

[I 2025-02-04 09:30:59,998] A new study created in RDB with name: 2025-02-04 09:30:59.314496


A new study created in RDB with name: 2025-02-04 09:30:59.314496


[I 2025-02-04 09:37:42,444] Trial 0 finished with value: 38.86869948781562 and parameters: {'num_leaves': 968, 'max_depth': 9, 'learning_rate': 0.0011665566671760525, 'n_estimators': 3672, 'min_child_samples': 196, 'subsample': 0.9530717440156706, 'colsample_bytree': 0.928390274422981, 'max_bin': 912, 'reg_alpha': 0.3518613622222736, 'reg_lambda': 5.970345448194738e-08, 'min_split_gain': 0.8549228988333699}. Best is trial 0 with value: 38.86869948781562.


Trial 0 finished with value: 38.86869948781562 and parameters: {'num_leaves': 968, 'max_depth': 9, 'learning_rate': 0.0011665566671760525, 'n_estimators': 3672, 'min_child_samples': 196, 'subsample': 0.9530717440156706, 'colsample_bytree': 0.928390274422981, 'max_bin': 912, 'reg_alpha': 0.3518613622222736, 'reg_lambda': 5.970345448194738e-08, 'min_split_gain': 0.8549228988333699}. Best is trial 0 with value: 38.86869948781562.


[I 2025-02-04 09:47:33,659] Trial 1 finished with value: 38.88870868328197 and parameters: {'num_leaves': 720, 'max_depth': 13, 'learning_rate': 0.00017031612944122965, 'n_estimators': 3949, 'min_child_samples': 131, 'subsample': 0.901330756793552, 'colsample_bytree': 0.8062240804290614, 'max_bin': 231, 'reg_alpha': 0.00722384973705176, 'reg_lambda': 1.201762108541826e-06, 'min_split_gain': 0.7907543047563852}. Best is trial 0 with value: 38.86869948781562.


...

Trial 97 finished with value: 38.85823873737133 and parameters: {'num_leaves': 462, 'max_depth': 8, 'learning_rate': 0.11744142155269753, 'n_estimators': 3839, 'min_child_samples': 167, 'subsample': 0.49269203503349257, 'colsample_bytree': 0.2738844638978022, 'max_bin': 925, 'reg_alpha': 0.04648028831629588, 'reg_lambda': 0.007423121018162321, 'min_split_gain': 0.5457907574384706}. Best is trial 54 with value: 38.85528038637746.


[I 2025-02-04 11:56:09,005] Trial 98 finished with value: 38.86005060620758 and parameters: {'num_leaves': 566, 'max_depth': 9, 'learning_rate': 0.059718583242816474, 'n_estimators': 3370, 'min_child_samples': 162, 'subsample': 0.44905401187390825, 'colsample_bytree': 0.25065659523152056, 'max_bin': 865, 'reg_alpha': 0.06773136791879975, 'reg_lambda': 0.002632361660319725, 'min_split_gain': 0.6105285462190055}. Best is trial 54 with value: 38.85528038637746.


Trial 98 finished with value: 38.86005060620758 and parameters: {'num_leaves': 566, 'max_depth': 9, 'learning_rate': 0.059718583242816474, 'n_estimators': 3370, 'min_child_samples': 162, 'subsample': 0.44905401187390825, 'colsample_bytree': 0.25065659523152056, 'max_bin': 865, 'reg_alpha': 0.06773136791879975, 'reg_lambda': 0.002632361660319725, 'min_split_gain': 0.6105285462190055}. Best is trial 54 with value: 38.85528038637746.


[I 2025-02-04 11:56:23,191] Trial 99 finished with value: 38.866753973653566 and parameters: {'num_leaves': 597, 'max_depth': 7, 'learning_rate': 0.09502731142012397, 'n_estimators': 3737, 'min_child_samples': 156, 'subsample': 0.5761245100511851, 'colsample_bytree': 0.3880804467286263, 'max_bin': 325, 'reg_alpha': 0.02360927180195827, 'reg_lambda': 0.00036950940641546667, 'min_split_gain': 0.5148135361744149}. Best is trial 54 with value: 38.85528038637746.


Trial 99 finished with value: 38.866753973653566 and parameters: {'num_leaves': 597, 'max_depth': 7, 'learning_rate': 0.09502731142012397, 'n_estimators': 3737, 'min_child_samples': 156, 'subsample': 0.5761245100511851, 'colsample_bytree': 0.3880804467286263, 'max_bin': 325, 'reg_alpha': 0.02360927180195827, 'reg_lambda': 0.00036950940641546667, 'min_split_gain': 0.5148135361744149}. Best is trial 54 with value: 38.85528038637746.


In [38]:
import os


'..//optuna/optuna_2025-02-04_12-18-00.log'

In [39]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': trial.suggest_int('num_leaves', 500, 1500),
        'max_depth': trial.suggest_int('max_depth', 6, 40),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 10000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 300),
        'subsample': trial.suggest_float('subsample', 0.05, .5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.15, .5),
        'max_bin': trial.suggest_int('max_bin', 500, 10000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.4, 0.6),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=200, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=100)

[I 2025-02-04 12:22:09,967] A new study created in RDB with name: 2025-02-04_12-22-09


A new study created in RDB with name: 2025-02-04_12-22-09
A new study created in RDB with name: 2025-02-04_12-22-09


[I 2025-02-04 12:22:35,814] Trial 0 finished with value: 38.83214048005187 and parameters: {'num_leaves': 1074, 'max_depth': 32, 'learning_rate': 0.1032184708260355, 'n_estimators': 9315, 'min_child_samples': 79, 'subsample': 0.2986549507351246, 'colsample_bytree': 0.37199977575137655, 'max_bin': 6605, 'reg_alpha': 5.280324603816464e-05, 'reg_lambda': 0.0006099973636033234, 'min_split_gain': 0.4179023899042439}. Best is trial 0 with value: 38.83214048005187.


Trial 0 finished with value: 38.83214048005187 and parameters: {'num_leaves': 1074, 'max_depth': 32, 'learning_rate': 0.1032184708260355, 'n_estimators': 9315, 'min_child_samples': 79, 'subsample': 0.2986549507351246, 'colsample_bytree': 0.37199977575137655, 'max_bin': 6605, 'reg_alpha': 5.280324603816464e-05, 'reg_lambda': 0.0006099973636033234, 'min_split_gain': 0.4179023899042439}. Best is trial 0 with value: 38.83214048005187.
Trial 0 finished with value: 38.83214048005187 and parameters: {'num_leaves': 1074, 'max_depth': 32, 'learning_rate': 0.1032184708260355, 'n_estimators': 9315, 'min_child_samples': 79, 'subsample': 0.2986549507351246, 'colsample_bytree': 0.37199977575137655, 'max_bin': 6605, 'reg_alpha': 5.280324603816464e-05, 'reg_lambda': 0.0006099973636033234, 'min_split_gain': 0.4179023899042439}. Best is trial 0 with value: 38.83214048005187.


[I 2025-02-04 12:23:52,519] Trial 1 finished with value: 38.84311497189168 and parameters: {'num_leaves': 1321, 'max_depth': 17, 'learning_rate': 0.014633949727060527, 'n_estimators': 6950, 'min_child_samples': 229, 'subsample': 0.08001932333696223, 'colsample_bytree': 0.36727582350679094, 'max_bin': 3813, 'reg_alpha': 0.3578773702293137, 'reg_lambda': 1.2083661735249292e-07, 'min_split_gain': 0.595273733777373}. Best is trial 0 with value: 38.83214048005187.


...

Trial 97 finished with value: 38.82369371597836 and parameters: {'num_leaves': 894, 'max_depth': 7, 'learning_rate': 0.08892931513394327, 'n_estimators': 3200, 'min_child_samples': 124, 'subsample': 0.16117079988877575, 'colsample_bytree': 0.3252084333944928, 'max_bin': 8952, 'reg_alpha': 0.005145855512415603, 'reg_lambda': 0.001051009076643046, 'min_split_gain': 0.49594462342344586}. Best is trial 24 with value: 38.8182533820295.
Trial 97 finished with value: 38.82369371597836 and parameters: {'num_leaves': 894, 'max_depth': 7, 'learning_rate': 0.08892931513394327, 'n_estimators': 3200, 'min_child_samples': 124, 'subsample': 0.16117079988877575, 'colsample_bytree': 0.3252084333944928, 'max_bin': 8952, 'reg_alpha': 0.005145855512415603, 'reg_lambda': 0.001051009076643046, 'min_split_gain': 0.49594462342344586}. Best is trial 24 with value: 38.8182533820295.


[I 2025-02-04 14:44:42,823] Trial 98 finished with value: 38.82271931582044 and parameters: {'num_leaves': 584, 'max_depth': 12, 'learning_rate': 0.07744885068793435, 'n_estimators': 4578, 'min_child_samples': 169, 'subsample': 0.3475112077505679, 'colsample_bytree': 0.30297806863994087, 'max_bin': 8496, 'reg_alpha': 0.03667967780065134, 'reg_lambda': 0.03078673169058207, 'min_split_gain': 0.5316743200146907}. Best is trial 24 with value: 38.8182533820295.


Trial 98 finished with value: 38.82271931582044 and parameters: {'num_leaves': 584, 'max_depth': 12, 'learning_rate': 0.07744885068793435, 'n_estimators': 4578, 'min_child_samples': 169, 'subsample': 0.3475112077505679, 'colsample_bytree': 0.30297806863994087, 'max_bin': 8496, 'reg_alpha': 0.03667967780065134, 'reg_lambda': 0.03078673169058207, 'min_split_gain': 0.5316743200146907}. Best is trial 24 with value: 38.8182533820295.
Trial 98 finished with value: 38.82271931582044 and parameters: {'num_leaves': 584, 'max_depth': 12, 'learning_rate': 0.07744885068793435, 'n_estimators': 4578, 'min_child_samples': 169, 'subsample': 0.3475112077505679, 'colsample_bytree': 0.30297806863994087, 'max_bin': 8496, 'reg_alpha': 0.03667967780065134, 'reg_lambda': 0.03078673169058207, 'min_split_gain': 0.5316743200146907}. Best is trial 24 with value: 38.8182533820295.


[I 2025-02-04 14:45:59,377] Trial 99 finished with value: 38.82701625776454 and parameters: {'num_leaves': 951, 'max_depth': 22, 'learning_rate': 0.022588125073006313, 'n_estimators': 2736, 'min_child_samples': 183, 'subsample': 0.13753723484627658, 'colsample_bytree': 0.28683798805687055, 'max_bin': 9232, 'reg_alpha': 5.2260673461966495e-06, 'reg_lambda': 1.2737699436646928e-05, 'min_split_gain': 0.5085495987127824}. Best is trial 24 with value: 38.8182533820295.


Trial 99 finished with value: 38.82701625776454 and parameters: {'num_leaves': 951, 'max_depth': 22, 'learning_rate': 0.022588125073006313, 'n_estimators': 2736, 'min_child_samples': 183, 'subsample': 0.13753723484627658, 'colsample_bytree': 0.28683798805687055, 'max_bin': 9232, 'reg_alpha': 5.2260673461966495e-06, 'reg_lambda': 1.2737699436646928e-05, 'min_split_gain': 0.5085495987127824}. Best is trial 24 with value: 38.8182533820295.
Trial 99 finished with value: 38.82701625776454 and parameters: {'num_leaves': 951, 'max_depth': 22, 'learning_rate': 0.022588125073006313, 'n_estimators': 2736, 'min_child_samples': 183, 'subsample': 0.13753723484627658, 'colsample_bytree': 0.28683798805687055, 'max_bin': 9232, 'reg_alpha': 5.2260673461966495e-06, 'reg_lambda': 1.2737699436646928e-05, 'min_split_gain': 0.5085495987127824}. Best is trial 24 with value: 38.8182533820295.


In [42]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': trial.suggest_int('num_leaves', 400, 2000),
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', .05, 0.15),
        'n_estimators': trial.suggest_int('n_estimators', 4000, 20000),
        'min_child_samples': trial.suggest_int('min_child_samples', 100, 300),
        'subsample': trial.suggest_float('subsample', 0.15, .25),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'max_bin': trial.suggest_int('max_bin', 500, 50000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, .05),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.5, 0.55),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=300, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=50)

[I 2025-02-04 15:51:05,800] A new study created in RDB with name: 2025-02-04_15-51-05


A new study created in RDB with name: 2025-02-04_15-51-05
A new study created in RDB with name: 2025-02-04_15-51-05
A new study created in RDB with name: 2025-02-04_15-51-05


[I 2025-02-04 15:52:53,341] Trial 0 finished with value: 38.80031675134514 and parameters: {'num_leaves': 1452, 'max_depth': 10, 'learning_rate': 0.0685323158402428, 'n_estimators': 12992, 'min_child_samples': 258, 'subsample': 0.21151700531236176, 'colsample_bytree': 0.34490035412327874, 'max_bin': 40237, 'reg_alpha': 0.025395274341882542, 'reg_lambda': 0.00021030313689250477, 'min_split_gain': 0.5106840686427828}. Best is trial 0 with value: 38.80031675134514.


Trial 0 finished with value: 38.80031675134514 and parameters: {'num_leaves': 1452, 'max_depth': 10, 'learning_rate': 0.0685323158402428, 'n_estimators': 12992, 'min_child_samples': 258, 'subsample': 0.21151700531236176, 'colsample_bytree': 0.34490035412327874, 'max_bin': 40237, 'reg_alpha': 0.025395274341882542, 'reg_lambda': 0.00021030313689250477, 'min_split_gain': 0.5106840686427828}. Best is trial 0 with value: 38.80031675134514.
Trial 0 finished with value: 38.80031675134514 and parameters: {'num_leaves': 1452, 'max_depth': 10, 'learning_rate': 0.0685323158402428, 'n_estimators': 12992, 'min_child_samples': 258, 'subsample': 0.21151700531236176, 'colsample_bytree': 0.34490035412327874, 'max_bin': 40237, 'reg_alpha': 0.025395274341882542, 'reg_lambda': 0.00021030313689250477, 'min_split_gain': 0.5106840686427828}. Best is trial 0 with value: 38.80031675134514.
Trial 0 finished with value: 38.80031675134514 and parameters: {'num_leaves': 1452, 'max_depth': 10, 'learning_rate': 0.06

[I 2025-02-04 15:54:34,417] Trial 1 finished with value: 38.807061309023155 and parameters: {'num_leaves': 1121, 'max_depth': 10, 'learning_rate': 0.0540403886175713, 'n_estimators': 16690, 'min_child_samples': 277, 'subsample': 0.20347339944559334, 'colsample_bytree': 0.2864741990559873, 'max_bin': 24185, 'reg_alpha': 0.06035319758802271, 'reg_lambda': 0.00026154063108341105, 'min_split_gain': 0.5212300347772861}. Best is trial 0 with value: 38.80031675134514.


...

Trial 47 finished with value: 38.7887382416345 and parameters: {'num_leaves': 1915, 'max_depth': 12, 'learning_rate': 0.05586030569092263, 'n_estimators': 19068, 'min_child_samples': 120, 'subsample': 0.197807098363482, 'colsample_bytree': 0.3059680989590157, 'max_bin': 38580, 'reg_alpha': 0.033103072351161826, 'reg_lambda': 7.208277423091038e-07, 'min_split_gain': 0.5331906545188253}. Best is trial 31 with value: 38.783956741666614.
Trial 47 finished with value: 38.7887382416345 and parameters: {'num_leaves': 1915, 'max_depth': 12, 'learning_rate': 0.05586030569092263, 'n_estimators': 19068, 'min_child_samples': 120, 'subsample': 0.197807098363482, 'colsample_bytree': 0.3059680989590157, 'max_bin': 38580, 'reg_alpha': 0.033103072351161826, 'reg_lambda': 7.208277423091038e-07, 'min_split_gain': 0.5331906545188253}. Best is trial 31 with value: 38.783956741666614.
Trial 47 finished with value: 38.7887382416345 and parameters: {'num_leaves': 1915, 'max_depth': 12, 'learning_rate': 0.0558

[I 2025-02-04 17:00:08,486] Trial 48 finished with value: 38.79793697606157 and parameters: {'num_leaves': 1026, 'max_depth': 11, 'learning_rate': 0.07349285739287606, 'n_estimators': 6736, 'min_child_samples': 226, 'subsample': 0.1723258936801395, 'colsample_bytree': 0.3150960977769042, 'max_bin': 42532, 'reg_alpha': 0.012614071976242624, 'reg_lambda': 0.014203938779004005, 'min_split_gain': 0.5279722375549875}. Best is trial 31 with value: 38.783956741666614.


Trial 48 finished with value: 38.79793697606157 and parameters: {'num_leaves': 1026, 'max_depth': 11, 'learning_rate': 0.07349285739287606, 'n_estimators': 6736, 'min_child_samples': 226, 'subsample': 0.1723258936801395, 'colsample_bytree': 0.3150960977769042, 'max_bin': 42532, 'reg_alpha': 0.012614071976242624, 'reg_lambda': 0.014203938779004005, 'min_split_gain': 0.5279722375549875}. Best is trial 31 with value: 38.783956741666614.
Trial 48 finished with value: 38.79793697606157 and parameters: {'num_leaves': 1026, 'max_depth': 11, 'learning_rate': 0.07349285739287606, 'n_estimators': 6736, 'min_child_samples': 226, 'subsample': 0.1723258936801395, 'colsample_bytree': 0.3150960977769042, 'max_bin': 42532, 'reg_alpha': 0.012614071976242624, 'reg_lambda': 0.014203938779004005, 'min_split_gain': 0.5279722375549875}. Best is trial 31 with value: 38.783956741666614.
Trial 48 finished with value: 38.79793697606157 and parameters: {'num_leaves': 1026, 'max_depth': 11, 'learning_rate': 0.073

[I 2025-02-04 17:01:45,240] Trial 49 finished with value: 38.800157961774595 and parameters: {'num_leaves': 1094, 'max_depth': 11, 'learning_rate': 0.08692143362305385, 'n_estimators': 4786, 'min_child_samples': 274, 'subsample': 0.20347285555782302, 'colsample_bytree': 0.3217891556352112, 'max_bin': 45446, 'reg_alpha': 0.016265864269621906, 'reg_lambda': 2.2229724886632678e-06, 'min_split_gain': 0.5131745753716289}. Best is trial 31 with value: 38.783956741666614.


Trial 49 finished with value: 38.800157961774595 and parameters: {'num_leaves': 1094, 'max_depth': 11, 'learning_rate': 0.08692143362305385, 'n_estimators': 4786, 'min_child_samples': 274, 'subsample': 0.20347285555782302, 'colsample_bytree': 0.3217891556352112, 'max_bin': 45446, 'reg_alpha': 0.016265864269621906, 'reg_lambda': 2.2229724886632678e-06, 'min_split_gain': 0.5131745753716289}. Best is trial 31 with value: 38.783956741666614.
Trial 49 finished with value: 38.800157961774595 and parameters: {'num_leaves': 1094, 'max_depth': 11, 'learning_rate': 0.08692143362305385, 'n_estimators': 4786, 'min_child_samples': 274, 'subsample': 0.20347285555782302, 'colsample_bytree': 0.3217891556352112, 'max_bin': 45446, 'reg_alpha': 0.016265864269621906, 'reg_lambda': 2.2229724886632678e-06, 'min_split_gain': 0.5131745753716289}. Best is trial 31 with value: 38.783956741666614.
Trial 49 finished with value: 38.800157961774595 and parameters: {'num_leaves': 1094, 'max_depth': 11, 'learning_rat

In [43]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': trial.suggest_int('num_leaves', 400, 2000),
        'max_depth': trial.suggest_int('max_depth', 6, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', .05, 0.15),
        'n_estimators': trial.suggest_int('n_estimators', 4000, 20000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'subsample': trial.suggest_float('subsample', 0.15, .25),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, .35),
        'max_bin': trial.suggest_int('max_bin', 50000, 200000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.01, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, .05),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.5, 0.55),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=300, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
storage_name = f"sqlite:///{os.path.join(optuna_dir, study_name)}.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='minimize')
study.optimize(objective, n_trials=50)

[I 2025-02-04 18:04:48,586] A new study created in RDB with name: 2025-02-04_18-04-48


A new study created in RDB with name: 2025-02-04_18-04-48
A new study created in RDB with name: 2025-02-04_18-04-48
A new study created in RDB with name: 2025-02-04_18-04-48
A new study created in RDB with name: 2025-02-04_18-04-48


[I 2025-02-04 18:07:02,563] Trial 0 finished with value: 38.79268120437156 and parameters: {'num_leaves': 900, 'max_depth': 7, 'learning_rate': 0.07402284243468071, 'n_estimators': 18766, 'min_child_samples': 180, 'subsample': 0.24130004303826283, 'colsample_bytree': 0.32471032025816254, 'max_bin': 103611, 'reg_alpha': 0.02755403498073013, 'reg_lambda': 7.591743151627048e-07, 'min_split_gain': 0.5049832980500218}. Best is trial 0 with value: 38.79268120437156.


Trial 0 finished with value: 38.79268120437156 and parameters: {'num_leaves': 900, 'max_depth': 7, 'learning_rate': 0.07402284243468071, 'n_estimators': 18766, 'min_child_samples': 180, 'subsample': 0.24130004303826283, 'colsample_bytree': 0.32471032025816254, 'max_bin': 103611, 'reg_alpha': 0.02755403498073013, 'reg_lambda': 7.591743151627048e-07, 'min_split_gain': 0.5049832980500218}. Best is trial 0 with value: 38.79268120437156.
Trial 0 finished with value: 38.79268120437156 and parameters: {'num_leaves': 900, 'max_depth': 7, 'learning_rate': 0.07402284243468071, 'n_estimators': 18766, 'min_child_samples': 180, 'subsample': 0.24130004303826283, 'colsample_bytree': 0.32471032025816254, 'max_bin': 103611, 'reg_alpha': 0.02755403498073013, 'reg_lambda': 7.591743151627048e-07, 'min_split_gain': 0.5049832980500218}. Best is trial 0 with value: 38.79268120437156.
Trial 0 finished with value: 38.79268120437156 and parameters: {'num_leaves': 900, 'max_depth': 7, 'learning_rate': 0.07402284

[I 2025-02-04 18:08:36,091] Trial 1 finished with value: 38.790845302392725 and parameters: {'num_leaves': 777, 'max_depth': 10, 'learning_rate': 0.10623985305056836, 'n_estimators': 12818, 'min_child_samples': 188, 'subsample': 0.1728888206430606, 'colsample_bytree': 0.25786429616592194, 'max_bin': 86960, 'reg_alpha': 0.01052849638568877, 'reg_lambda': 4.5037316336291763e-08, 'min_split_gain': 0.5285121838734385}. Best is trial 1 with value: 38.790845302392725.


...

Trial 34 finished with value: 38.783407135239656 and parameters: {'num_leaves': 949, 'max_depth': 10, 'learning_rate': 0.05553665806620914, 'n_estimators': 7266, 'min_child_samples': 21, 'subsample': 0.234052006796841, 'colsample_bytree': 0.317512514706488, 'max_bin': 122505, 'reg_alpha': 0.4753253374738805, 'reg_lambda': 1.7260409184022813e-08, 'min_split_gain': 0.5426641518825144}. Best is trial 27 with value: 38.77232789938401.
Trial 34 finished with value: 38.783407135239656 and parameters: {'num_leaves': 949, 'max_depth': 10, 'learning_rate': 0.05553665806620914, 'n_estimators': 7266, 'min_child_samples': 21, 'subsample': 0.234052006796841, 'colsample_bytree': 0.317512514706488, 'max_bin': 122505, 'reg_alpha': 0.4753253374738805, 'reg_lambda': 1.7260409184022813e-08, 'min_split_gain': 0.5426641518825144}. Best is trial 27 with value: 38.77232789938401.
Trial 34 finished with value: 38.783407135239656 and parameters: {'num_leaves': 949, 'max_depth': 10, 'learning_rate': 0.055536658

[I 2025-02-04 19:26:59,058] Trial 35 finished with value: 38.78092250921433 and parameters: {'num_leaves': 573, 'max_depth': 9, 'learning_rate': 0.06279108645092474, 'n_estimators': 9834, 'min_child_samples': 33, 'subsample': 0.23999679538309665, 'colsample_bytree': 0.3104950455201615, 'max_bin': 169663, 'reg_alpha': 0.35869723407696896, 'reg_lambda': 0.002257558490496617, 'min_split_gain': 0.5478424609391683}. Best is trial 27 with value: 38.77232789938401.


Trial 35 finished with value: 38.78092250921433 and parameters: {'num_leaves': 573, 'max_depth': 9, 'learning_rate': 0.06279108645092474, 'n_estimators': 9834, 'min_child_samples': 33, 'subsample': 0.23999679538309665, 'colsample_bytree': 0.3104950455201615, 'max_bin': 169663, 'reg_alpha': 0.35869723407696896, 'reg_lambda': 0.002257558490496617, 'min_split_gain': 0.5478424609391683}. Best is trial 27 with value: 38.77232789938401.
Trial 35 finished with value: 38.78092250921433 and parameters: {'num_leaves': 573, 'max_depth': 9, 'learning_rate': 0.06279108645092474, 'n_estimators': 9834, 'min_child_samples': 33, 'subsample': 0.23999679538309665, 'colsample_bytree': 0.3104950455201615, 'max_bin': 169663, 'reg_alpha': 0.35869723407696896, 'reg_lambda': 0.002257558490496617, 'min_split_gain': 0.5478424609391683}. Best is trial 27 with value: 38.77232789938401.
Trial 35 finished with value: 38.78092250921433 and parameters: {'num_leaves': 573, 'max_depth': 9, 'learning_rate': 0.06279108645

[I 2025-02-04 19:29:17,617] Trial 36 finished with value: 38.786777460325695 and parameters: {'num_leaves': 842, 'max_depth': 11, 'learning_rate': 0.05272503424468774, 'n_estimators': 5276, 'min_child_samples': 49, 'subsample': 0.2324610805899151, 'colsample_bytree': 0.32707196855020787, 'max_bin': 151085, 'reg_alpha': 0.17559253697047825, 'reg_lambda': 0.01073652310067165, 'min_split_gain': 0.5382202721670754}. Best is trial 27 with value: 38.77232789938401.


Trial 36 finished with value: 38.786777460325695 and parameters: {'num_leaves': 842, 'max_depth': 11, 'learning_rate': 0.05272503424468774, 'n_estimators': 5276, 'min_child_samples': 49, 'subsample': 0.2324610805899151, 'colsample_bytree': 0.32707196855020787, 'max_bin': 151085, 'reg_alpha': 0.17559253697047825, 'reg_lambda': 0.01073652310067165, 'min_split_gain': 0.5382202721670754}. Best is trial 27 with value: 38.77232789938401.
Trial 36 finished with value: 38.786777460325695 and parameters: {'num_leaves': 842, 'max_depth': 11, 'learning_rate': 0.05272503424468774, 'n_estimators': 5276, 'min_child_samples': 49, 'subsample': 0.2324610805899151, 'colsample_bytree': 0.32707196855020787, 'max_bin': 151085, 'reg_alpha': 0.17559253697047825, 'reg_lambda': 0.01073652310067165, 'min_split_gain': 0.5382202721670754}. Best is trial 27 with value: 38.77232789938401.
Trial 36 finished with value: 38.786777460325695 and parameters: {'num_leaves': 842, 'max_depth': 11, 'learning_rate': 0.0527250

## Older stuff

In [27]:
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'random_state': 42,
        # Narrowed ranges for faster convergence (you can tweak these)
        'num_leaves': trial.suggest_int('num_leaves', 3, 512),
        'max_depth': trial.suggest_int('max_depth', 1, 16),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-2, 0.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'max_bin': trial.suggest_int('max_bin', 63, 1011),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
    
    gbm = lgb.train(
        params=param,
        train_set=train_data,
        num_boost_round=500,
        valid_sets=[train_data, valid_data],
        valid_names=['train_0', 'valid_0'],
        callbacks=[
            LightGBMPruningCallback(trial, "rmse", valid_name="valid_0"),
            lgb.log_evaluation(-1)                   # Suppress training logs
        ]
    )
    
    preds = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))
    
    return rmse

# 6. Create and run the study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2025-02-04 06:34:58,549] A new study created in memory with name: no-name-0ef73315-47fe-4229-b8f0-975463622cd5
[I 2025-02-04 06:35:18,737] Trial 0 finished with value: 38.85835965643685 and parameters: {'num_leaves': 275, 'max_depth': 7, 'learning_rate': 0.06747451694850155, 'min_child_samples': 48, 'subsample': 0.7301614143973025, 'colsample_bytree': 0.5627798598992705, 'max_bin': 793, 'reg_alpha': 0.00011220199806930794, 'reg_lambda': 0.010719331890043265, 'min_split_gain': 0.4956674404440702}. Best is trial 0 with value: 38.85835965643685.
[I 2025-02-04 06:35:25,357] Trial 1 finished with value: 38.87059928486902 and parameters: {'num_leaves': 321, 'max_depth': 9, 'learning_rate': 0.16928805209180173, 'min_child_samples': 12, 'subsample': 0.6643571810688592, 'colsample_bytree': 0.9405091772003791, 'max_bin': 600, 'reg_alpha': 3.861419362485028e-08, 'reg_lambda': 0.003795269932834587, 'min_split_gain': 0.360626343629453}. Best is trial 0 with value: 38.85835965643685.
[I 2025-02-0

In [26]:
study.trials_dataframe().sort_values(by='value').head(30)

,number,value,datetime_start,datetime_complete,duration,params_colsample_bytree,params_learning_rate,params_max_bin,params_max_depth,params_min_child_samples,params_min_split_gain,params_num_leaves,params_reg_alpha,params_reg_lambda,params_subsample,state
51,51,38.857316,2025-02-03 20:45:31.759361,2025-02-03 20:45:40.181372,0 days 00:00:08.422011,0.642800,0.313805,802,7,5,0.693125,35,3.032549e-01,4.755506e-06,0.813096,COMPLETE
19,19,38.858180,2025-02-03 20:44:00.169796,2025-02-03 20:44:10.500697,0 days 00:00:10.330901,0.702409,0.167917,878,13,28,0.823958,63,2.573974e-02,2.990846e-05,0.746294,COMPLETE
18,18,38.858228,2025-02-03 20:43:50.120216,2025-02-03 20:44:00.168949,0 days 00:00:10.048733,0.811278,0.166753,889,6,46,0.819260,59,2.257147e-02,7.944908e-06,0.749822,COMPLETE
20,20,38.858274,2025-02-03 20:44:10.502200,2025-02-03 20:44:20.783032,0 days 00:00:10.280832,0.695166,0.174654,911,13,6,0.798209,75,1.321920e-01,7.901545e-06,0.738470,COMPLETE
4,4,38.859070,2025-02-03 20:42:35.161132,2025-02-03 20:43:03.045022,0 days 00:00:27.883890,0.977706,0.031785,861,9,82,0.607971,219,2.388832e-02,1.286258e-03,0.911795,COMPLETE
3,3,38.859165,2025-02-03 20:42:21.791843,2025-02-03 20:42:35.160101,0 days 00:00:13.368258,0.956149,0.085596,663,5,51,0.468909,27,2.978180e-05,7.372884e-08,0.601102,COMPLETE
34,34,38.859293,2025-02-03 20:45:04.769022,2025-02-03 20:45:11.885661,0 days 00:00:07.116639,0.646438,0.314709,801,7,5,0.684384,37,3.722760e-01,3.789686e-06,0.799644,COMPLETE
21,21,38.859438,2025-02-03 20:44:20.784334,2025-02-03 20:44:27.454948,0 days 00:00:06.670614,0.701982,0.177975,933,14,7,0.845954,60,1.417497e-01,9.642180e-06,0.748612,COMPLETE
11,11,38.859898,2025-02-03 20:43:29.660756,2025-02-03 20:43:39.797529,0 days 00:00:10.136773,0.991250,0.105884,720,8,44,0.730256,10,7.011859e-03,2.729079e-08,0.507301,COMPLETE
52,52,38.859912,2025-02-03 20:45:40.182278,2025-02-03 20:45:46.285322,0 days 00:00:06.103044,0.690738,0.390571,765,6,10,0.773912,18,2.187565e-01,1.077200e-06,0.906104,COMPLETE


In [23]:
study.trials_dataframe().sort_values(by='value')

,number,value,datetime_start,datetime_complete,duration,params_colsample_bytree,params_learning_rate,params_max_bin,params_max_depth,params_min_child_samples,params_min_split_gain,params_num_leaves,params_reg_alpha,params_reg_lambda,params_subsample,state
3,3,38.862396,2025-02-03 20:19:13.666658,2025-02-03 20:19:26.246400,0 days 00:00:12.579742,0.620356,0.116175,460,7,75,0.342268,297,7.137420e-03,1.437058e-06,0.699740,COMPLETE
16,16,38.863771,2025-02-03 20:19:44.287852,2025-02-03 20:19:51.860448,0 days 00:00:07.572596,0.596050,0.198295,335,6,43,0.484206,313,1.332847e-08,3.037763e-07,0.691041,COMPLETE
6,6,38.864721,2025-02-03 20:19:30.776301,2025-02-03 20:19:37.832819,0 days 00:00:07.056518,0.605384,0.197061,242,4,69,0.620968,321,6.966264e-08,4.596502e-06,0.769576,COMPLETE
28,28,38.865196,2025-02-03 20:20:14.071051,2025-02-03 20:20:22.903044,0 days 00:00:08.831993,0.531971,0.197729,240,5,70,0.351519,409,4.232914e-08,1.908976e-05,0.776353,COMPLETE
2,2,38.866191,2025-02-03 20:18:52.041135,2025-02-03 20:19:13.665922,0 days 00:00:21.624787,0.658276,0.027538,146,8,37,0.091179,363,4.007225e-05,6.170344e-03,0.697801,COMPLETE
23,23,38.866999,2025-02-03 20:19:55.276131,2025-02-03 20:20:03.623879,0 days 00:00:08.347748,0.500204,0.195059,181,6,46,0.365529,178,7.023048e-08,7.414473e-06,0.734897,COMPLETE
26,26,38.867098,2025-02-03 20:20:04.767388,2025-02-03 20:20:13.522828,0 days 00:00:08.755440,0.778924,0.152462,461,8,58,0.645709,247,3.439508e-06,4.884874e-08,0.737739,COMPLETE
31,31,38.868725,2025-02-03 20:20:24.154022,2025-02-03 20:20:27.379304,0 days 00:00:03.225282,0.539390,0.197010,225,5,70,0.355715,416,4.602589e-08,2.334629e-05,0.781634,PRUNED
1,1,38.869808,2025-02-03 20:18:40.220186,2025-02-03 20:18:52.040437,0 days 00:00:11.820251,0.878914,0.081455,79,7,72,0.394845,274,8.386588e-02,3.059195e-01,0.982106,COMPLETE
38,38,38.873332,2025-02-03 20:20:30.789099,2025-02-03 20:20:36.076933,0 days 00:00:05.287834,0.975496,0.195595,306,8,40,0.538047,496,2.046494e-03,2.420219e-02,0.622010,PRUNED


In [28]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        # 'boosting_type': boosting_type,
        'num_leaves': trial.suggest_int('num_leaves', 200, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 5000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 200),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, 1.0),
        'max_bin': trial.suggest_int('max_bin', 50, 1000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.1, 1.0),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    # For the DART boosting type, include dart‑specific parameters.

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=100, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse


study_1 = optuna.create_study(direction='minimize')
study_1.optimize(objective, n_trials=100)

[I 2025-02-04 06:39:57,522] A new study created in memory with name: no-name-8779507c-c261-46cc-990e-13fb4a513a66
[I 2025-02-04 06:45:00,895] Trial 0 finished with value: 38.883937593360734 and parameters: {'num_leaves': 597, 'max_depth': 7, 'learning_rate': 0.000364113256801268, 'n_estimators': 3158, 'min_child_samples': 103, 'subsample': 0.6115069297497427, 'colsample_bytree': 0.821486379484551, 'max_bin': 635, 'reg_alpha': 0.6733018374807459, 'reg_lambda': 2.2533816042611835e-08, 'min_split_gain': 0.2289803907588847}. Best is trial 0 with value: 38.883937593360734.
[I 2025-02-04 06:47:14,772] Trial 1 finished with value: 38.86480244436043 and parameters: {'num_leaves': 906, 'max_depth': 7, 'learning_rate': 0.00840845676199378, 'n_estimators': 3956, 'min_child_samples': 58, 'subsample': 0.7497712653298377, 'colsample_bytree': 0.3357351243363299, 'max_bin': 75, 'reg_alpha': 0.03758166053679331, 'reg_lambda': 1.685842589203576e-08, 'min_split_gain': 0.3951629800119736}. Best is trial 1

In [ ]:
study.trials_dataframe().sort_values(by='value')

In [10]:
def objective(trial):    
    # Define the hyperparameter search space
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        # 'boosting_type': boosting_type,
        'num_leaves': trial.suggest_int('num_leaves', 200, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 50, 5000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 200),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, 1.0),
        'max_bin': trial.suggest_int('max_bin', 50, 1000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.1, 1.0),
        'random_state': 42,
        'early_stopping_rounds': 50,
        'verbosity': -1,
    }
    
    # For the DART boosting type, include dart‑specific parameters.

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    # Initialize the model with these parameters
    model = lgb.train(param, train_data, num_boost_round=100, valid_sets=[valid_data])
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))    
    return rmse


study_1 = optuna.create_study(direction='minimize')
study_1.optimize(objective, n_trials=100)

[I 2025-02-02 21:19:40,729] A new study created in memory with name: no-name-bb8f98fc-1f6a-41b4-a14b-b1a863e2abf9
[I 2025-02-02 22:01:37,642] Trial 0 finished with value: 39.76996423248435 and parameters: {'boosting_type': 'dart', 'num_leaves': 337, 'max_depth': 4, 'learning_rate': 0.0031635309200795584, 'n_estimators': 4372, 'min_child_samples': 48, 'subsample': 0.34645324972913083, 'colsample_bytree': 0.6577425331360448, 'max_bin': 872, 'reg_alpha': 3.5368612289072567, 'reg_lambda': 0.01035388459360509, 'min_split_gain': 0.6966281128935405, 'drop_rate': 0.17004025288994346}. Best is trial 0 with value: 39.76996423248435.
[I 2025-02-02 22:03:00,806] Trial 1 finished with value: 54.86826168805091 and parameters: {'boosting_type': 'dart', 'num_leaves': 640, 'max_depth': 3, 'learning_rate': 0.0017345753663105348, 'n_estimators': 1426, 'min_child_samples': 116, 'subsample': 0.3843193222965111, 'colsample_bytree': 0.813550191319027, 'max_bin': 109, 'reg_alpha': 0.007101289855733979, 'reg_l

In [13]:
study_1.trials_dataframe().sort_values(by='value')

,number,value,datetime_start,datetime_complete,duration,params_boosting_type,params_colsample_bytree,params_drop_rate,params_learning_rate,params_max_bin,params_max_depth,params_min_child_samples,params_min_split_gain,params_n_estimators,params_num_leaves,params_reg_alpha,params_reg_lambda,params_subsample,state
95,95,38.905450,2025-02-02 23:24:09.866394,2025-02-02 23:24:15.822479,0 days 00:00:05.956085,gbdt,0.332398,NaN,0.005068,926,5,30,0.650490,3441,700,4.723607,2.456838e+00,0.845381,COMPLETE
98,98,38.905666,2025-02-02 23:24:27.850445,2025-02-02 23:24:31.026854,0 days 00:00:03.176409,gbdt,0.332299,NaN,0.010603,819,5,37,0.645073,2565,705,1.251035,1.164912e+00,0.906303,COMPLETE
99,99,38.905702,2025-02-02 23:24:31.027556,2025-02-02 23:24:35.744511,0 days 00:00:04.716955,gbdt,0.340497,NaN,0.007422,799,5,36,0.658667,2602,698,4.132585,1.151869e+00,0.850854,COMPLETE
74,74,38.905933,2025-02-02 23:11:57.621138,2025-02-02 23:12:09.566929,0 days 00:00:11.945791,gbdt,0.312253,NaN,0.002252,974,4,27,0.560021,3595,745,5.654588,7.114641e-01,0.788073,COMPLETE
77,77,38.906371,2025-02-02 23:17:04.959326,2025-02-02 23:17:17.460252,0 days 00:00:12.500926,gbdt,0.280696,NaN,0.002499,883,5,30,0.632505,3360,735,4.152447,2.182133e-01,0.750337,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,6,53.032265,2025-02-02 22:29:47.879299,2025-02-02 22:36:53.092748,0 days 00:07:05.213449,dart,0.995450,0.442523,0.001402,162,10,195,0.688799,1812,247,0.008998,7.218142e-06,0.722218,COMPLETE
1,1,54.868262,2025-02-02 22:01:37.643474,2025-02-02 22:03:00.806160,0 days 00:01:23.162686,dart,0.813550,0.289182,0.001735,109,3,116,0.745261,1426,640,0.007101,5.889920e-08,0.384319,COMPLETE
39,39,69.640643,2025-02-02 22:49:05.528314,2025-02-02 22:52:46.171398,0 days 00:03:40.643084,dart,0.303269,0.384219,0.000694,192,10,34,0.884639,1271,500,0.000488,5.689784e-06,0.630937,COMPLETE
76,76,90.602337,2025-02-02 23:12:19.653034,2025-02-02 23:17:04.958682,0 days 00:04:45.305648,dart,0.317347,0.088862,0.003573,942,4,26,0.556991,3591,679,2.040453,1.984707e+00,0.705245,COMPLETE


In [ ]:
print("Best trial:")
best_trial = study_1.best_trial
print("  RMSE: {:.4f}".format(best_trial.value))
print("  Best hyperparameters:")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

Index(['id', 'Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price'],
      dtype='object')